In [2]:
from bs4 import BeautifulSoup

with open('home.html', 'r') as html_file:
    content = html_file.read()

    # content of the html file, parser method we want to use
    soup = BeautifulSoup(content, 'lxml')
    tags = soup.find('h5')  # searches for the first element
    courses_html_tags = soup.find_all('h5')

    # Display all the courses
    for course in courses_html_tags:
        # We'll get only the text for each element in the list returned by soup.find_all
        print(course.text)


Python for beginners
Python Web Development
Python Machine Learning


In [51]:
# Grab the prices

from bs4 import BeautifulSoup

with open ('home.html','r') as html_file:
    content = html_file.read()
    # print(content)
    soup = BeautifulSoup(content, 'lxml')
    # class is a built-in method in Python, that's why we need to use class_
    course_cards = soup.find_all('div',class_='card' )
    for course in course_cards:
        course_name = course.h5.text
        course_price = course.a.text.split()[-1] # to access the last element, which will be the price
        
        # course.a or course.h5 can access different tags inside a div and returns a tag
        
        print(f"The course {course_name} costs {course_price}")
    
    

The course Python for beginners costs 20$
The course Python Web Development costs 50$
The course Python Machine Learning costs 100$


In [31]:
print(help(str.split))

Help on method_descriptor:

split(self, /, sep=None, maxsplit=-1) unbound builtins.str method
    Return a list of the substrings in the string, using sep as the separator string.

      sep
        The separator used to split the string.

        When set to None (the default value), will split on any whitespace
        character (including \n \r \t \f and spaces) and will discard
        empty strings from the result.
      maxsplit
        Maximum number of splits.
        -1 (the default value) means no limit.

    Splitting starts at the front of the string and works to the end.

    Note, str.split() is mainly useful for data that has been intentionally
    delimited.  With natural text that includes punctuation, consider using
    the regular expression module.

None


-----BeautifulSoup constructor-----


Parameters:
markup (required):
The string or file-like object containing the HTML or XML content you want to parse. This can be:

A string of HTML or XML

An open file or URL (with .read() called or passed directly)

parser (optional but recommended):
The parser you want BeautifulSoup to use. Common choices:

"html.parser" — built-in Python HTML parser (default)

"lxml" — faster, requires the lxml library

"xml" — for parsing XML with lxml

"html5lib" — parses like a web browser, very lenient, requires html5lib

listings is a ResultSet (a special list-like object from BeautifulSoup).

Each item in listings (e.g., listing) is a Tag.

Tag objects have the .find(), .find_all(), .text, etc. methods.

.get() to access any attribute of a Tag element

In [ ]:
%pip install lxml

# Scrape real websites using requests

from bs4 import BeautifulSoup
import requests
from datetime import datetime
def find_jobs():
    html_text = requests.get(
        'https://m.timesjobs.com/mobile/jobs-search-result.html?txtKeywords=python&cboWorkExp1=-1&txtLocation=')
    soup = BeautifulSoup(html_text.text, 'lxml')
    listings = soup.find_all('div', class_='srp-listing')


    for listing in listings:
        job_link = listing.find('a').get('href')
        html_job_text = requests.get(job_link).text
        inner_soup = BeautifulSoup(html_job_text, 'lxml')
        outer_infos = inner_soup.find('div', class_=['jd-page', 'ui-page', 'ui-page-theme-a',
                                                    'ui-page-header-fixed', 'ui-page-footer-fixed', 'ui-page-active'])
        if outer_infos:
            inner_infos = outer_infos.find(
                'div', class_='jdpage-main')
            # Get the location and the years of experience
            location_exp_infos = inner_infos.find('div', class_='clearfix exp-loc')
            location_list = location_exp_infos.find(
                'div', class_='srp-loc jd-loc').text.split()

            print(location_list[1].strip('()/') +
                ' - ' + location_list[2].strip('()/'))

            years_of_experience = location_exp_infos.find(
                'div', class_='srp-exp').text.split()
            print(years_of_experience[0] + ' ' + years_of_experience[1])

            # Get the job title, the company and the day the job listing was posted
            job_information = inner_infos.find('div', id='jobTitle')
            job_title = job_information.h1.text
            company_name = job_information.h2.span.text
            posting_time = job_information.find('span', class_='posting-time').text
            posting_time_date = datetime.strptime(posting_time,'%d %b, %Y').date() #b means abbreviated time
            # posting_time_date = posting_time_date.strftime("%d-%m-%Y")
            print(job_title + " "+company_name+" "+str(posting_time_date))
            print('------------------------------------------------------')
            
if __name__ == '__main__':
    find_jobs()
            

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?